Exercise 4

In [9]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import time
import numba
import pylab

# to print plots inline
%matplotlib inline

In [10]:
# Specify parameters
# firms
alpha_k = 0.29715
alpha_l = 0.65
delta = 0.154
psi = 1.08
z = 1
sigma_z = 0.213
mu = 0.000
rho = 0.7605
sizez = 9

# households
h = 6.616
beta = 0.96

r_bar = 1/beta - 1

params = alpha_k, alpha_l, delta, psi, h, beta, r_bar

In [11]:
# Set up K-grid
dens = 2
# put in bounds here for the capital stock space
betafirm = 1/(1 + r_bar)
kstar = ((((1 / betafirm - 1 + delta) * ((0.7 / alpha_l) **
                                         (alpha_l / (1 - alpha_l)))) /
         (alpha_k * (z ** (1 / (1 - alpha_l))))) **
         ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 2 * kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
for j in range(int(numb * dens)):
    K[j] = ub_k * (1 - delta) ** (j / dens)
kvec = K[::-1]
sizek = kvec.shape[0]

import sys
sys.path.append('../Code')
import ar1_approx as ar1
num_sigma = 4
step = (num_sigma * sigma_z) / (sizez / 2)
pi, zvec = ar1.rouwen(rho, mu, step, sizez)
zvec = np.exp(zvec)

In [14]:
# operating profits, op
def op(z, k, w, params):
    alpha_k, alpha_l, delta, psi, h, beta, r_bar = params
    op = ((1 - alpha_l) * ((alpha_l / w) ** (alpha_l / (1 - alpha_l))) * ((k ** alpha_k) ** (1 / (1 - alpha_l)))) * z ** (1/(1 - alpha_l))
    return op

# firm cash flow, e
@numba.jit
def flow(w, params):
    e = np.zeros((sizek, sizek, sizez))
    for i in range(sizek):
        for j in range(sizek):
            for k in range(sizez):
                e[i, j, k] = (op(zvec[k], kvec[i], w, params) - kvec[j] + ((1 - delta) * kvec[i]) - 
                              ((psi / 2) * ((kvec[j] - ((1 - delta) * kvec[i])) ** 2)/ kvec[i]))
    return e

In [19]:
@numba.jit
def VFI(w_bar, params):
    alpha_k, alpha_l, delta, psi, h, beta, r_bar = params
    VFtol = 1e-6
    VFdist = 7.0
    VFmaxiter = 3000
    V = np.zeros((sizek, sizez))  # initial guess of value function
    Vmat = np.zeros((sizek, sizek, sizez))  # initialize Vmat matrix
    VFiter = 1
    
    while VFdist > VFtol and VFiter < VFmaxiter:
        TV = V
        for i in range(sizek):  # loop over k
            for j in range(sizek):  # loop over k'
                for k in range(sizez): # loop over z
                    Vmat[i, j, k] = flow(w_bar, params)[i, j, k] + betafirm * np.dot(V[j, :], pi[:, k])
    
    # iteration for graphing later
    V = Vmat.max(axis=1)  # apply max operator to Vmat (to get V(k))
    PF = np.argmax(Vmat, axis=1)  # find the index of the optimal k'
    VFdist = (np.absolute(V - TV)).max()  # check distance between value
    # function for this iteration and value function from past iteration
    VFiter += 1
    optK = kvec[PF]
    optI = optK - (1 - delta) * optK
    return V, PF, optK, optI

In [ ]:
@numba.jit
def firm(w_guess, params):
    alpha_k, alpha_l, delta, psi, h, beta, r_bar = params
    V, PF, optK, optI = VFI(w_guess, params)
    
    for i in range(sizek):
        e = flow(w_guess, params)
        ld = 
        ind = opt[I] - (1 - delta) 
    